In [9]:
import numpy as np
import pandas as pd
import plotly.express as px

In [10]:
import json
from dataclasses import dataclass
from typing import List, Dict, Any

#TODO ENUM
FENCE = 'fence'
MY_SNAKE = '_snake'
MY_SNAKE_HEAD = '_snake_head'
ENEMY_SNAKE = 'enemy'
ENEMY_SNAKE_HEAD = 'enemy_head'
FOOD_USUAL = 'food'
FOOD_GOLDEN  = 'golden'
FOOD_SUSP = 'susp'


ALIVE = 'alive'
DEAD = 'dead'
UNKNOWN = 'unknown'

@dataclass
class State:
    point_type: str
    #TODO class
    x: int
    y: int
    z: int
    #
    points_or_kills: int
    death_count: int
    revive_sec: int
    status: str
    #TODO class
    direction_x: int
    direction_y: int
    direction_z: int
    #
    turn: int
    custom: Any = None

In [3]:
def get_status(status):
    if status == 'alive': return ALIVE
    if status == 'dead': return DEAD
    return UNKNOWN

In [11]:


class Vizual:
    def __init__(self, data_points, data_states):
        self.data_points = data_points
        self.data_states = data_states
        self.make_info()

    @classmethod
    def from_file(class_, file):
        points = []
        static_states = []
        with open(file, 'r') as f:
            for line in f.readlines():
                data_points, static_state = class_.read_state(json.loads(line))
                points.extend(data_points)
                static_states.append(static_state)

        return class_(points, static_states)
    
    @staticmethod
    def read_state(data):
        static = {
            'name': data['name'],
            'map_size': data['mapSize'],
            'team_points': data['points'],
            'turn': data['turn'],
            'reviveTimeoutSec': data['reviveTimeoutSec'], # ???
            'tickRemainMs': data['tickRemainMs'], # ???
            'errors': str(data['errors']), # ???
        }
        turn = data['turn']
        points = []
        for fence in data['fences']:
            points.append(
                State(
                    FENCE, 
                    x=fence[0],
                    y=fence[1],
                    z=fence[2],
                    points_or_kills=0,
                    death_count=0,
                    revive_sec=0,
                    status=ALIVE,
                    direction_x=0,
                    direction_y=0,
                    direction_z=0,
                    turn = turn,
                ).__dict__
            )
        for snake in data['snakes']:
            is_head = True
            for (x,y,z) in snake['geometry']:
                points.append(
                    State(
                        snake['id'][:10]+(MY_SNAKE_HEAD if is_head else MY_SNAKE),
                        x=x,
                        y=y,
                        z=z,
                        points_or_kills=0,
                        death_count=snake['deathCount'],
                        revive_sec=snake['reviveRemainMs'],
                        status=get_status(snake['status']),
                        direction_x=snake['direction'][0],
                        direction_y=snake['direction'][1],
                        direction_z=snake['direction'][2],
                        turn = turn,
                    ).__dict__
                )
                is_head = False
        
        for enemy in data['enemies']:
            is_head = True
            for (x,y,z) in enemy['geometry']:
                points.append(
                    State(
                        (ENEMY_SNAKE_HEAD if is_head else ENEMY_SNAKE),
                        x=x,
                        y=y,
                        z=z,
                        points_or_kills=enemy['kills'],
                        death_count=0,
                        revive_sec=0,
                        status=get_status(enemy['status']),
                        direction_x=0,
                        direction_y=0,
                        direction_z=0,
                        turn = turn,
                    ).__dict__
                )
                is_head = False

        for food in data['food']:
            x,y,z = food['c']
            points.append(
                State(
                    FOOD_USUAL,
                    x=x,
                    y=y,
                    z=z,
                    points_or_kills=food['points'],
                    death_count=0,
                    revive_sec=0,
                    status=ALIVE,
                    direction_x=0,
                    direction_y=0,
                    direction_z=0,
                    turn = turn,
                ).__dict__
            )
        return points, static

    def make_info(self):
        self.df_data = pd.DataFrame(self.data_points)
        self.df_data['size'] = 1
        self.x_max, self.y_max, self.z_max = self.data_states[0]['map_size']
        self.team_name = self.data_states[0]['name']
        
        pass

    def show(self, height=1200, width=1200, start=0, end=3000):
        fig = px.scatter_3d(self.df_data.loc[(lambda x: (x.status == ALIVE)&(x.turn >= start)&(x.turn <= end))].sort_values('turn'), 
                    x='x', 
                    y='y', 
                    z='z',
                    color='point_type',
                    title='Gamecon',
                    animation_frame='turn',
                    hover_data = ['point_type', 'x', 'y', 'z', 'points_or_kills', 'death_count', 'revive_sec', 'status', 'direction_x', 'direction_y', 'direction_z'],
                    range_x=[0, self.x_max],
                    range_y=[0, self.y_max],
                    range_z=[0, self.z_max],
                    size='size',
                    height=height,
                    width=width)
        return fig

In [5]:
v = Vizual.from_file('round3-logs')

In [ ]:
v.show(start=300, end=1000)

In [53]:
from pathlib import Path

def list_all_files(directory):
    # Use rglob('*') to get all files recursively
    all_files = [str(file) for file in Path(directory).rglob('*/response*') if file.is_file()]
    return all_files

# Example usage
directory_path = './snake3d-day1-3'
files = list_all_files(directory_path)


with open('./round3-logs', 'w') as wf:
    for file in files:
        with open(file) as f:
            wf.write(json.dumps(json.load(f))+'\n')

with open('./round3-logs') as wf:
    print(len(wf.readlines()))

1529


In [8]:
v.df_data.loc[lambda x: (x.turn == 452)&(x.point_type.str.contains('snake_head'))]

,point_type,x,y,z,points_or_kills,death_count,revive_sec,status,direction_x,direction_y,direction_z,turn,custom,size
551931,d1f3cb9144_snake_head,115,143,4,0,1,0,alive,0,0,-1,452,None,1
551932,0adb74c3a3_snake_head,0,108,25,0,0,0,alive,0,0,-1,452,None,1
551939,9b63718d0e_snake_head,1,24,23,0,2,0,alive,0,0,-1,452,None,1


In [69]:
v.df_data.loc[lambda x: (x.turn == 85)&(x.point_type.str.contains('snake_head'))]

,point_type,x,y,z,points_or_kills,death_count,revive_sec,status,direction_x,direction_y,direction_z,turn,custom
3890369,d1f3cb9144_snake_head,52,43,31,0,0,0,alive,0,1,0,85,None
3890372,0adb74c3a3_snake_head,79,114,39,0,0,0,alive,0,-1,0,85,None


In [70]:
df = pd.DataFrame(v.data_states).sort_values('turn')

In [71]:
df['team_points_prev'] = df['team_points'].shift().fillna(0).astype(int)

In [72]:
df[(df.team_points < df.team_points_prev)&(df.team_points != 0)]

,name,map_size,team_points,turn,reviveTimeoutSec,tickRemainMs,errors,team_points_prev
372,VeKaVa,"[180, 180, 60]",98,52,60,66,['snake 9b63718d0ea1f3ec541f3705b9db2b6ba9eae5...,282
341,VeKaVa,"[180, 180, 60]",74,63,60,435,['snake 9b63718d0ea1f3ec541f3705b9db2b6ba9eae5...,290
1237,VeKaVa,"[180, 180, 60]",67,73,60,399,['snake 9b63718d0ea1f3ec541f3705b9db2b6ba9eae5...,74
404,VeKaVa,"[180, 180, 60]",25,77,60,284,['snake 9b63718d0ea1f3ec541f3705b9db2b6ba9eae5...,67
243,VeKaVa,"[180, 180, 60]",25,96,60,420,['snake 9b63718d0ea1f3ec541f3705b9db2b6ba9eae5...,290
580,VeKaVa,"[180, 180, 60]",138,156,60,274,[],466
367,VeKaVa,"[180, 180, 60]",419,322,60,642,[],563
187,VeKaVa,"[180, 180, 60]",389,346,60,839,['snake d1f3cb9144c857532c5d767ab441a9153a8743...,432
1472,VeKaVa,"[180, 180, 60]",350,427,60,738,[],389
61,VeKaVa,"[180, 180, 60]",315,455,60,620,['snake d1f3cb9144c857532c5d767ab441a9153a8743...,350


In [79]:
px.line(df.loc[lambda x: x.team_points != 0], x='turn', y=['team_points'])